In [17]:
import torch
import gpytorch as gp
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

torch.manual_seed(1)
np.random.seed(1)

In [18]:
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

In [19]:
class MyGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module  = gp.means.ZeroMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel())

    def forward(self, x):
        mean_x  = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [20]:
for n_train in ([32, 64, 128]):
    for lr in ([0.001, 0.01, 0.02, 0.05]):

        # TRAIN SET
        Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[1: n_train+1]
        Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[0: n_train]
        Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
        Tsa_k2   = np.array(df['supply_discharge_temp'])[2: n_train+2]
        valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[2: n_train+2]

        train_x = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
        train_x = torch.tensor(train_x, dtype=torch.float32)
        train_y = torch.tensor(Tsa_k2, dtype=torch.float32)

        # TEST SET
        Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[n_train+1: 1600+1]
        Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train: 1600]
        Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: 1600+1]
        Tsa_k2   = np.array(df['supply_discharge_temp'])[n_train+2: 1600+2]
        valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[n_train+2: 1600+2]

        test_x = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
        test_x = torch.tensor(test_x, dtype=torch.float32)
        test_y = torch.tensor(Tsa_k2, dtype=torch.float32)

        # initialize likelihood and model
        likelihood = gp.likelihoods.GaussianLikelihood()
        model      = MyGPModel(train_x, train_y, likelihood)

        # Training
        training_iter = 200000    # number of training iteration

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)       # optimizer
        mll = gp.mlls.ExactMarginalLogLikelihood(likelihood, model)   # marginal likelihood

        R2_test = np.array([], dtype=np.float32)
        for i in range(training_iter+1):
            # TRAIN
            model.train()                   # find the hyperparameters
            likelihood.train()

            optimizer.zero_grad()           # Zero gradients from previous iteration
            output = model(train_x)         # Output from model
            loss   = -mll(output, train_y)  # Calc loss and backprop gradients

            loss.backward()
            optimizer.step()
            
            # EVAL
            model.eval()
            likelihood.eval()

            with torch.no_grad(), gp.settings.fast_pred_var():
                pred_y = likelihood(model(test_x))

            r2 = r2_score(test_y.numpy(), pred_y.mean.numpy())
            R2_test = np.append(R2_test, r2)
        
        # print(np.argmax(R2_test))
        print('n:%4d \t lr:%5.3f \t r2:%8.4f' % (n_train, lr, np.max(R2_test)))

torch.Size([32])
torch.Size([32, 1])


RuntimeError: grad can be implicitly created only for scalar outputs